# Pytorch Lightning Framework for training S+Z Galaxy Classifiers

## Imports

In [ ]:
import os
from enum import Enum
import pandas as pd
import torch
from torch.utils.data import random_split
import lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger,CSVLogger
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
import albumentations as A
from galaxy_datasets.pytorch.galaxy_datamodule import GalaxyDataModule
from ChiralityClassifier import ChiralityClassifier

## Options

In [ ]:
class modes(Enum):
    FULL_DATASET = 0 #Use all 600,000 galaxies
    CUT_DATASET = 1 #Use cut of 200,000 galaxies, with pre-selected test data and downsampled train data
    BEST_SUBSET = 2 #Select N best S,Z & other galaxies, evenly split
    LOCAL_SUBSET = 3 #Use local cache of 1500 galaxies

IMG_SIZE = 160 # This is the output size of the generated image array
MODE = modes.CUT_DATASET #Select which dataset to run
TEST_PRETRAINED_MODEL = True #Load and run testing dataset & save metrics
# Models:
#resnet18,resnet34,resnet50,resnet101,resnet152,
#jiaresnet50,LeNet,G_ResNet18,G_LeNet,
MODEL_NAME = "G_ResNet18"
CUSTOM_ID = "save_test"

USE_TENSORBOARD = False #Log to tensorboard as well as csv logger
SAVE_MODEL = False #Save model weights to .pt file
REPEAT_RUNS = 1 #Set to 1 for 1 run

METRICS_PATH = "../Metrics"
#SAVE_PATH = "../Models" #In process of updating - will move to metrics
LOG_PATH = "../Code/lightning_logs"
FULL_DATA_PATH = "/share/nas2/walml/galaxy_zoo/decals/dr8/jpg"
LOCAL_SUBSET_DATA_PATH = "../Data/Subset"

FULL_CATALOG_PATH = "../Data/gz1_desi_cross_cat.csv"
CUT_CATALOG_TEST_PATH = "../Data/gz1_desi_cross_cat_testing.csv"
CUT_CATALOG_TRAIN_PATH = "../Data/gz1_desi_cross_cat_train_val_downsample.csv"
BEST_SUBSET_CATALOG_PATH = "../Data/gz1_desi_cross_cat_best_subset.csv"
LOCAL_SUBSET_CATALOG_PATH = "../Data/gz1_desi_cross_cat_local_subset.csv"

torch.set_float32_matmul_precision("medium")
if len(CUSTOM_ID) == 0:
    MODEL_ID = f"{MODEL_NAME}_{MODE.name.lower()}"
else:
     MODEL_ID = f"{MODEL_NAME}_{MODE.name.lower()}_{CUSTOM_ID}"
PRETRAINED_MODEL_PATH = f"{METRICS_PATH}/{MODEL_ID}/version_{0}/model.pt"

## GPU Test

In [ ]:
print(f"Using pytorch {torch.__version__}. CPU cores available on device: {os.cpu_count()}")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type == "cuda":
    print(torch.cuda.get_device_name(0))
    print(f"Allocated Memory: {round(torch.cuda.memory_allocated(0)/1024**3,1)} GB")
    print(f"Cached Memory: {round(torch.cuda.memory_reserved(0)/1024**3,1)} GB")
print("Using device:", device)

## Reading in data

### Building catalog

In [ ]:
def get_file_paths(catalog_to_convert,folder_path):
    brick_ids = catalog_to_convert["dr8_id"].str.split("_",expand=True)[0]
    dr8_ids = catalog_to_convert["dr8_id"]
    file_locations = folder_path+"/"+brick_ids+"/"+dr8_ids+".jpg"
    print(f"Created {file_locations.shape[0]} galaxy filepaths")
    return file_locations

def check_folder(save_dir):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

def generate_transforms(resize_after_crop=IMG_SIZE):
    transforms_to_apply = [
        A.ToFloat(), #Converts from 0-255 to 0-1

        A.Resize( #Resizes to 160x160
            height=resize_after_crop,
            width=resize_after_crop,
            interpolation=1,
            always_apply=True
        ),

        #A.randomrotate
    ]
    return A.Compose(transforms_to_apply)

In [ ]:
if MODE == modes.CUT_DATASET:
    train_val_catalog = pd.read_csv(CUT_CATALOG_TRAIN_PATH)
    train_val_catalog["file_loc"] = get_file_paths(train_val_catalog,FULL_DATA_PATH)
    generator1 = torch.Generator().manual_seed(42) #Preset test-val split, note test dataloader will still shuffle
    train_catalog, val_catalog = random_split(train_val_catalog, [0.20,0.80], generator=generator1)
    train_catalog = train_catalog.dataset.iloc[train_catalog.indices]
    val_catalog = val_catalog.dataset.iloc[val_catalog.indices]   
    test_catalog = pd.read_csv(CUT_CATALOG_TEST_PATH)
    test_catalog["file_loc"] = get_file_paths(test_catalog,FULL_DATA_PATH)

    datamodule = GalaxyDataModule(
        label_cols=["P_CW","P_ACW","P_OTHER"],
        train_catalog=train_catalog, val_catalog=train_catalog, test_catalog=test_catalog,
        custom_albumentation_transform=generate_transforms(),
        batch_size=200,
        num_workers=11,
    )
    
else:
    if MODE == modes.FULL_DATASET:
        catalog = pd.read_csv(FULL_CATALOG_PATH)
        catalog["file_loc"] = get_file_paths(catalog,FULL_DATA_PATH)

    elif MODE == modes.BEST_SUBSET:
        catalog = pd.read_csv(BEST_SUBSET_CATALOG_PATH)
        catalog["file_loc"] = get_file_paths(catalog,FULL_DATA_PATH)

    elif MODE == modes.LOCAL_SUBSET:
        catalog = pd.read_csv(LOCAL_SUBSET_CATALOG_PATH)
        catalog["file_loc"] = get_file_paths(catalog,LOCAL_SUBSET_DATA_PATH)

    datamodule = GalaxyDataModule(
        label_cols=["P_CW","P_ACW","P_OTHER"],
        catalog=catalog,
        train_fraction=0.7, val_fraction=0.15, test_fraction=0.15,
        custom_albumentation_transform=generate_transforms(),
        batch_size=100,
        num_workers=11,
    )

## Code to run

In [ ]:
datamodule.prepare_data()
datamodule.setup()

In [ ]:
for run in range(0,REPEAT_RUNS):
    
    save_dir = f"{METRICS_PATH}/{MODEL_ID}/version_{run}"
    check_folder(save_dir)

    model = ChiralityClassifier(
        num_classes=(2 if (MODEL_NAME=="jiaresnet50") else 3), #2 for Jia et al version
        model_version=MODEL_NAME,
        optimizer="adamw",
        scheduler  ="steplr",
        lr=0.0001,
        weight_decay=0,
        step_size=5,
        gamma=0.85,
        batch_size=60,
        weights=(PRETRAINED_MODEL_PATH if TEST_PRETRAINED_MODEL else None),
        model_save_path=f"{save_dir}/model.pt",
        graph_save_path=f"{save_dir}/matrix.png"
    )

    tb_logger = TensorBoardLogger(LOG_PATH, name=MODEL_ID,version=run)
    csv_logger = CSVLogger(LOG_PATH,name=MODEL_ID,version=run)

    trainer = pl.Trainer(
        accelerator=("gpu" if device.type=="cuda" else "cpu"),
        max_epochs=1,
        devices=1,
        logger=([tb_logger,csv_logger] if USE_TENSORBOARD else csv_logger),
        default_root_dir=f"{LOG_PATH}/{MODEL_ID}"
        #profiler="pytorch"
        #callbacks=EarlyStopping(monitor="val_loss", mode="min")
    )

    #compiled_model = torch.compile(model, backend="eager")
    

    if not TEST_PRETRAINED_MODEL:
        trainer.fit(model,train_dataloaders=datamodule.train_dataloader(),val_dataloaders=datamodule.val_dataloader())
        trainer.test(model,dataloaders=datamodule.val_dataloader())

        if SAVE_MODEL:
            torch.save(trainer.model.state_dict(), model.model_save_path)
            
    else:
        trainer.test(model,dataloaders=datamodule.test_dataloader())    

In [ ]:
state = torch.load(PRETRAINED_MODEL_PATH)

for key in state.keys():
    print(key)